In [3]:
import glob
import re

import numpy as np
import pandas as pd
import tqdm
import itertools

from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
# pd.options.display.max_columns = 2000

<ipython-input-3-287819f2dada>:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [4]:
class IteratorLogs():

    def __init__(self, logfile) -> None:
        with open(logfile) as f:
            self.lines = f.readlines()
        self.index = 0

    def next(self):
        temp = self.next_line()
        # filter GLOG
        while temp.find('.h:') != -1 or temp.find('.cc:') != -1:
            temp = self.next_line()
        # print(self.index, temp)
        return temp

    def next_line(self):
        if self.index == len(self.lines):
            raise StopIteration()
        temp = self.lines[self.index]
        self.index += 1
        return temp


# run_log_file to [{ts: ts1, k1: v, k2: v}, (ts2, {})]
def parse_log(logfile, isclient=False):
    iter_log = IteratorLogs(logfile)

    results = []
    dic = dict()

    i = 0
    while True:
        try:
            each_line = iter_log.next()
            each_line = each_line.strip()
            
            r = re.search(r'\[proc (\d)\] 8000 steps, sample: (.*), forward: (.*), backward: (.*), update: (.*)', each_line)
            if r:
                proc = int(r[1])
                dic[f"{proc}_sample"] = float(r[2])
                dic[f"{proc}_forward"] = float(r[3])
                dic[f"{proc}_backward"] = float(r[4])
                dic[f"{proc}_update"] = float(r[5])
                continue
            
            results.append(dic)
            ts = None
            dic = dict()
        except StopIteration:
            # if 'ts' in dic.keys():
            #     results.append(dic)
            break

    return results